In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [2]:
import pandas as pd
import re
import numpy as np
import itertools

In [6]:
header= ["레시피일련번호",'레시피제목','요리명','등록자ID','등록자명','조회수','추천수','스크랩수','요리방법별명','요리상황별명','요리재료별명','요리종류별명','요리소개','요리재료내용','요리인분명','요리난이도명',"요리시간명","최초등록일시"]
ckg = pd.read_csv('만개의레시피.csv', header=None, names=header, encoding='cp949')
ckg.drop(0, inplace=True)
ckg.info()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,5,6,7,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 128400 entries, 1 to 128400
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  128400 non-null  object
 1   레시피제목    128400 non-null  object
 2   요리명      126310 non-null  object
 3   등록자ID    128400 non-null  object
 4   등록자명     128394 non-null  object
 5   조회수      128400 non-null  object
 6   추천수      128400 non-null  object
 7   스크랩수     128400 non-null  object
 8   요리방법별명   128400 non-null  object
 9   요리상황별명   127510 non-null  object
 10  요리재료별명   128400 non-null  object
 11  요리종류별명   128400 non-null  object
 12  요리소개     127783 non-null  object
 13  요리재료내용   127340 non-null  object
 14  요리인분명    124171 non-null  object
 15  요리난이도명   128137 non-null  object
 16  요리시간명    120054 non-null  object
 17  최초등록일시   128400 non-null  object
dtypes: object(18)
memory usage: 18.6+ MB


In [7]:
ckg.drop(['등록자ID','등록자명','요리인분명','요리난이도명',"요리시간명","최초등록일시"], axis=1, inplace=True)
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
1,128671,어묵김말이,어묵김말이,9592,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
2,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
3,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
4,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
5,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...


In [8]:
#요리재료내용 결측치 삭제
ckg_drop = ckg[ckg.loc[:,'요리재료내용'].isnull()].index
ckg_drop = ckg.drop(ckg_drop)
ckg_drop.reset_index(drop=True, inplace=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127340 entries, 0 to 127339
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   레시피일련번호  127340 non-null  object
 1   레시피제목    127340 non-null  object
 2   요리명      125389 non-null  object
 3   조회수      127340 non-null  object
 4   추천수      127340 non-null  object
 5   스크랩수     127340 non-null  object
 6   요리방법별명   127340 non-null  object
 7   요리상황별명   126451 non-null  object
 8   요리재료별명   127340 non-null  object
 9   요리종류별명   127340 non-null  object
 10  요리소개     126859 non-null  object
 11  요리재료내용   127340 non-null  object
dtypes: object(12)
memory usage: 11.7+ MB


In [9]:
ckg_drop.요리종류별명.value_counts()

밑반찬         25346
메인반찬        20343
밥/죽/떡       17447
빵           11834
국/탕         10404
면/만두         8585
디저트          6457
찌개           4092
김치/젓갈/장류     3855
기타           3690
과자           3393
차/음료/술       2863
양식           2796
샐러드          2746
양념/소스/잼      2099
퓨전            820
스프            570
Name: 요리종류별명, dtype: int64

In [10]:
ckg_drop.요리재료별명.value_counts()

채소류       33865
해물류       14102
밀가루       12692
가공식품류     12136
돼지고기       7698
달걀/유제품     7540
소고기        5763
쌀          5597
닭고기        5372
기타         5263
과일류        4328
콩/견과류      4205
건어물류       3735
버섯류        1713
육류         1712
곡류         1619
Name: 요리재료별명, dtype: int64

In [11]:
#디저트, 차/음료/술 , 양념/소스/잼, 과자, 빵 카테고리 제외
drop_list = ckg_drop.loc[(ckg_drop['요리종류별명']=='디저트')
                        | (ckg_drop['요리종류별명']=='양념/소스/잼')
                        | (ckg_drop['요리종류별명']=='빵')
                        | (ckg_drop['요리종류별명']=='과자')
                        | (ckg_drop['요리종류별명']=='차/음료/술')
                        | (ckg_drop['요리종류별명']=='샐러드')
                        | (ckg_drop['요리종류별명']=='스프')].index
ckg_drop.drop(drop_list, inplace=True)
ckg_drop.reset_index(drop=True, inplace=True)
ckg_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97378 entries, 0 to 97377
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   레시피일련번호  97378 non-null  object
 1   레시피제목    97378 non-null  object
 2   요리명      95875 non-null  object
 3   조회수      97378 non-null  object
 4   추천수      97378 non-null  object
 5   스크랩수     97378 non-null  object
 6   요리방법별명   97378 non-null  object
 7   요리상황별명   96911 non-null  object
 8   요리재료별명   97378 non-null  object
 9   요리종류별명   97378 non-null  object
 10  요리소개     97018 non-null  object
 11  요리재료내용   97378 non-null  object
dtypes: object(12)
memory usage: 8.9+ MB


In [12]:
ckg_drop.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용
0,128892,두부에 꼬리가 달렸어요!!,NaN,5538,3,26,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
1,128932,입안에서 톡톡톡,NaN,6802,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
2,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
3,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
4,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...


## 요리 재료 전처리

In [ ]:
###재료 전처리

#요리재료_전처리(new column)
ckg_drop['요리재료_전처리'] = np.NaN
one_len = ['무','파','꿀','깨','떡','햄','콩','빵','닭','밤','면','잣','쌈', '마', '쑥','굴']

for idx in range(len(ckg_drop)):
    ingredient = ckg_drop.loc[idx, '요리재료내용']
    ingredient = re.sub(r'\[(.*?)\]', '|', ingredient)
    ingredient = re.sub(r'\([^)]*\)|[^ㄱ-ㅣ가-힣 \|]', ' ', str(ingredient)) #한국어 제외 모두 제거

    ingredient = ingredient.strip()
    ingredients = ingredient.split('|')

    #한자리 요리 재료 제외한 나머지 제거
    ingredient_list = []
    for ingredient in ckg_drop['요리재료_전처리']:
        ingredients = ingredient.split(', ')
        
        for i in ingredients:
            if (len(i) == 1) and (i not in one_len):
                pass
            else:
                ingredient_list.extend(i)

    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        i = i.split(' ')

        if len(i) == 1 and i != None : #계량 단위 없는 경우
            ingre_lists.append(i)
        elif len(i) > 1:
            i.pop(-1)    #가장 마지막의 계량 단위 제거
            i = ' '.join(i)
            i = [i.strip()]
            ingre_lists.append(i)
        else :
            pass

    #차원 축소
    ingre_lists = list(itertools.chain(*ingre_lists))
    
    #null 제거
    ingre_lists = list(filter(None, ingre_lists))
    
    #요리재료_전처리(new column)에 추가
    ckg_drop.loc[idx, '요리재료_전처리'] = ', '.join(ingre_lists)

ckg_drop.iloc[10000:10005,:]

In [ ]:
ckg_drop.to_csv('ckg_drop.csv')

## 재료 전체 사전을 만들기 위한 작업

In [ ]:
#한자리 요리 재료 제외한 나머지 제거
one_len = ['무','파','꿀','깨','떡','햄','콩','빵','닭','밤','면','잣','쌈', '마', '쑥','굴']

ingredient_list = []
for ingredient in ckg_drop['요리재료_전처리']:
    ingredients = ingredient.split(', ')
    ingredient_list.extend(ingredients)

In [ ]:
#한자리 요리 재료 제외한 나머지 제거
one_len = ['무','파','꿀','깨','떡','햄','콩','빵','닭','밤','면','잣','쌈', '마', '쑥','굴']

ingredient_list = []
for ingredient in ckg_drop['요리재료_전처리']:
    ingredients = ingredient.split(', ')
    
    for i in ingredients:
        if (len(i) == 1) and (i not in one_len):
            pass
        else:
            ingredient_list.extend(i)


In [ ]:

#쿠팡 밀키트에만 있는 재료 추가
coupang_only = ['샤브용육수', '숙면','전골용 육수','곱창비법','오뗄팜S','소 스지', '전각 슬라이스','해물탕 양념',
                '유자폰즈','소간장소스', '샤브용육수','유탕면','돼지고기 민찌','얼큰짬뽕소스', '오징어건더기',
                '명태곤이','된장국양념','꽃게된장찌개용 양념', '바지락맛 액상소스','해물탕 양념','유탕면','조피볼락',
                '진종합', '순살가마볼', '고추볼', '한모듬가마보꼬', '순살가마보꼬', '간사이원형', '어묵만두','가마보꼬바',
                '프레스 햄','짬뽕 소스','쥬키니 호박','야채 믹스','수비드찜닭','사골농축액','사골 농축액','찹햄',
                '코테기노','파개장 오일', '파개장 소스','문치가자미','문치가자미','스테이크 씨즈닝','돈마호크','오향보쌈',
                '보쌈무','대두유','우동 면','올리브 오일','크러쉬드 페퍼', '사천 고추', '허브 솔트','훈제 파프리카 파우더',
                '감자 옹심이','해물 엑기스','두반장 소스','단호박 칼국수','양장피 면','새꼬막살','수비드 찜닭']
ingredient_list.extend(coupang_only)

In [ ]:
#중복값 제거
ingredient_list = list(set(ingredient_list))
ingredient_list.pop(0)
ingredient_list

In [ ]:
print([i for i in ingredient_list if len(i) == 1])

In [ ]:
#텍스트 파일로 전체 재료 저장(중복x)
with open('만개의레시피_재료_목록.txt', 'w') as f :
    for ingre in ingredient_list:
        f.write(ingre+',')
len(ingredient_list)

37969

In [ ]:
def countLetters(word):
    counter = {}
    for letter in word:
        if letter not in counter:
            counter[letter]=0
        counter[letter] +=1
    return counter

#dict1 = countLetters(ingerdient_list2)

In [ ]:
#오타 검수를 위한 리스트 생성(중복o)
ingredient_list2 = []
for ingredient in ckg_drop['요리재료_전처리']:
    ingredients = ingredient.split(' ')
    ingredient_list2.extend(ingredients)

#차원축소
#ingredient_list2 = np.concatenate(ingredient_list2).tolist()

#null값 제거
ingredient_list2 = list(filter(None, ingredient_list2))
ingredient_list2

In [ ]:
#텍스트 파일로 전체 재료 저장(중복o)
import pickle
with open('만개의레시피_재료_목록_중복o.pickle', 'wb') as fw :
    pickle.dump(ingredient_list2, fw)
len(ingredient_list2)

951650